In [3]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import DensityMatrix

# 1. 创建量子电路（示例：1比特H门+X门，2比特CX门）
qc1 = QuantumCircuit(1)
qc1.h(0)  # 1比特叠加态电路
qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)  # 2比特纠缠态电路

# 2. 计算密度矩阵（直接传入电路）
rho1 = DensityMatrix(qc1)  # 1比特密度矩阵（2×2）
rho2 = DensityMatrix(qc2)  # 2比特密度矩阵（4×4）

# 3. 查看结果（可输出矩阵形式或简化信息）
print("1比特电路的密度矩阵：")
print(rho1.data)  # 输出复数矩阵（numpy数组）
print("\n2比特纠缠态的密度矩阵（前2行前2列）：")
print(rho2.data)  # 简化输出（4×4矩阵较大）

1比特电路的密度矩阵：
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]

2比特纠缠态的密度矩阵（前2行前2列）：
[[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]


In [4]:
from dataloader import get_qc_cvae_dataloader

In [5]:
# 测试配置：小样本量+少拓扑类型，快速验证
test_config = {
    "batch_size": 2,
    "base_num_samples": 3,
    "num_qubits": 4,
    "max_depth": 8,
    "topo_types": ["star", "ring"],  # 测试已修复的 star 和新增的 ring 拓扑
    "basic_gates": ['h', 'x', 'cx', 'swap'],
    "regenerate": False,  # 首次运行设为 True 生成样本
    "shuffle": False
}

# 创建数据加载器
dataloader = get_qc_cvae_dataloader(**test_config)
print(f"\n数据加载器创建完成，批次总数：{len(dataloader)}")

Transpile参数组合数：19

数据加载器创建完成，批次总数：57


In [6]:
data = next(iter(dataloader))

In [7]:
data.keys()

dict_keys(['g', 't', 'g_star', 'quantum_origin', 'quantum_optimized', 'optimization_metrics', 'circuit_origin_info', 'circuit_optimized_info', 'meta'])

In [8]:
g = data['g']
g

DataBatch(x=[2, 8], edge_index=[0], batch=[2], ptr=[3])

In [9]:
g.num_graphs

2

In [10]:
g.num_node_features

8

In [11]:
g.x.shape

torch.Size([2, 8])

In [12]:
g.edge_index

tensor([], dtype=torch.int64)

In [13]:
circuit_origin_info = data['circuit_origin_info']

In [14]:
circuit_origin_info

{'num_qubits': tensor([4, 4]),
 'total_gates': tensor([1, 1]),
 'depth': tensor([1, 1]),
 'two_qubit_gates': tensor([0, 0])}

In [24]:
data['circuit_optimized_info']

{'num_qubits': tensor([4, 4]),
 'total_gates': tensor([1, 1]),
 'depth': tensor([1, 1]),
 'two_qubit_gates': tensor([0, 0])}

In [26]:
data['quantum_optimized']

{'statevector': tensor([[0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
          0.0000+0.j, 0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
          0.0000+0.j, 0.0000+0.j],
         [0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
          0.0000+0.j, 0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j,
          0.0000+0.j, 0.0000+0.j]]),
 'unitary': tensor([[[ 0.7071+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
            0.0000+0.j,  0.0000+0.j,  0.7071+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
            0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
          [ 0.0000+0.j,  0.7071+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
            0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.7071+0.j,  0.0000+0.j,  0.0000+0.j,
            0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
          [ 0.0000+0.j,  0.0000+0.j,  0.70

In [25]:
data['optimization_metrics']

{'fidelity': tensor([1., 1.]),
 'depth_ratio': tensor([1., 1.]),
 'total_gate_ratio': tensor([1., 1.]),
 'two_qubit_ratio': tensor([nan, nan])}

In [15]:
from dataloader import data_to_quantum_circuit

In [20]:
qc = data_to_quantum_circuit(g[0], num_qubits=4, gate_types=['h', 'x', 'cx', 'swap'])

In [22]:
qc.draw()

q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ H ├
     └───┘

In [23]:
qco = data_to_quantum_circuit(data['g_star'][0],
                               num_qubits=4, gate_types=['h', 'x', 'cx', 'swap'])
qco.draw()

q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ H ├
     └───┘